<a href="https://colab.research.google.com/github/Zelechos/Generador-Css/blob/main/GeneradorCSS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto Generador de CSS [v0.0.4]**

In [ ]:
import torch
import torch.nn as nn

# Clonamos nuestro repositorio en donde se encuentra nuestro dataset con este comando

In [ ]:
!git clone https://github.com/Zelechos/Generador-Css.git

In [ ]:
# en este caso no es necesario ya que tenemos el dataset en github por ende tenemos que trabajar con el dataset y ver como esta organizado el json del inge
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#hacemos una copia para trabajar con el dataset
import shutil

shutil.copy("/content/drive/MyDrive/revisiones.json","/content/revisiones.json")

# Realizamos el Preprocesamiento de Textos

In [ ]:
import json

with open('/content/revisiones.json') as file:
    data = json.load(file)

#accedemos a la evaluation
print("evaluation : ", data["paper"][0]["review"][0]["evaluation"])

#accedemos a la text
print("text : ", data["paper"][0]["review"][0]["text"])

#averiguando la longitud
print("longitud : ", len(data["paper"][0]["review"]))

paper = data["paper"]


# Iniciamos con el Transformer

In [ ]:
class SelfAttention(nn.Module):
  def __init__(self, embed_size, heads):
    super(SelfAttention, self).__init__()
    self.embed_size = embed_size
    self.heads = heads
    self.head_dim = embed_size // heads

    assert (self.head_dim * heads == embed_size), "Embed size needs to be div by heads"

    self.values = nn.Linear(self.head_dim, self.head_dim, bias=False)
    self.keys = nn.Linear(self.head_dim, self.head_dim, bias=False)
    self.queries = nn.Linear(self.head_dim, self.head_dim, bias=False)
    self.fc_out = nn.Linear(heads * self.head_dim, embed_size)

  def forward(self, values, keys, query, mask):
    N = query.shape[0]
    value_len, key_len, query_len =  values.shape[1], keys.shape[1], query.shape[1]

    # dividir el incrustando self.heads en pedazos
    values = values.reshape(N, value_len, self.heads, self.head_dim)  
    keys = keys.reshape(N, key_len, self.heads, self.head_dim)  
    queries = query.reshape(N, key_len, self.heads, self.head_dim)  

    energy = torch.einsum("nqhd, nkhd->nhqk", [queries, keys])


    if mask is not None :
      energy = energy.masked_fill(mask == 0, float("-1e20"))
    
    
    attention = torch.softmax(energy / (self.embed_size ** (1/2)), dim = 3)

    out = torch.einsum("nhql,nlhd->nqhd", [attention, values]).reshape(
      N, query_len, self.heads * self.head_dim        
    )

    out = self.fc_out(out)
    return out


# Clase TransformerBlock

In [ ]:
class TransformerBlock()
